<a href="https://colab.research.google.com/github/smnieee/ml_workshop/blob/master/session5/1_FunWithNeuralNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fun with Neural Networks

---

## Southern Minnesota IEEE: Machine Learning Workshop 5
## May 16, 2022

In [ ]:
from tensorflow import keras
from keras import layers

In [ ]:
mnist = keras.datasets.mnist